In [13]:
import pandas as pd
import numpy as np
import datetime
import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

## Data Preprocessing

In [80]:
stocks = pd.read_json(r"data.json")

In [81]:
def preprocess_dataset (stocks):
    stocks.rename({"priceCurrencyOne":"priceUSD"}, axis = 1, inplace = True)
    stocks.rename({"priceCurrencyTwo":"priceEUR"}, axis = 1, inplace = True)

    stocks.drop(["currencyOne", "currencyTwo"], axis = 1, inplace = True)
    return stocks

def extract_currency (stocks, currency):
    return stocks[(stocks['tmsp']!='null')
                &(stocks['priceUSD']!='null')
                &(stocks['priceEUR']!='null')
               &(stocks['cryptoCurrency']==currency)]

def preprocess_currency (sample):
    sample.tmsp = [datetime.datetime.fromtimestamp(x) for x in sample.tmsp.astype(int)]
    sample.priceUSD = sample.priceUSD.astype(float)
    sample.priceEUR = sample.priceEUR.astype(float)
    return sample

In [82]:
stocks = preprocess_dataset(stocks)
sample_BTC = extract_currency(stocks, "BTC")
sample_BTC = preprocess_currency(sample_BTC)

## Features

In [83]:
sample_BTC["diff_eur"] = (sample_BTC.priceEUR/sample_BTC.priceEUR.shift(1))-1
sample_BTC["diff_usd"] = (sample_BTC.priceUSD/sample_BTC.priceUSD.shift(1))-1

## Parameter tuning

In [84]:
def list_pop (my_list):
    my_list_new = my_list.copy()
    return list(filter(lambda a: a != "wait", my_list_new))[-1]

def trading_func (timespan, sensitivity, elements_to_process, data):
    final_list = [[-1, 'placeholder']]
    
    for i in tqdm.tqdm(list(range(elements_to_process))): #53549
        sample_BTC_new = data[(data.tmsp>=(data.tmsp[i]- pd.to_timedelta(f"00:00:{timespan}"))) 
                              & (data.tmsp<=data.tmsp[i])].copy()
        value = ((sample_BTC_new[sample_BTC_new.tmsp == sample_BTC_new.tmsp.max()].priceUSD.values[0])/sample_BTC_new[sample_BTC_new.tmsp== sample_BTC_new.tmsp.min()].priceUSD.values[0])-1 

        if ((abs(value) > sensitivity)) and value>0:
            if final_list[-1][1] != 'buy':
                final_list.append([i,"buy"]) 

        elif ((abs(value) > sensitivity)) and value<0: 
            if final_list[-1][1] != 'sell':
                final_list.append([i, "sell"]) 
                
    final_list.remove([-1, 'placeholder'])
    return final_list

def calc_profit (sample_BTC):
    basic_value = sample_BTC.priceEUR[0]
    sample_BTC["profit"] = 0
    sample_BTC["profit"] = np.where(sample_BTC.decision.isin(["buy", "sell"]), (sample_BTC.priceEUR/basic_value)-1, sample_BTC.profit)
    return sample_BTC

In [86]:
for t in tqdm.tqdm([15, 20, 30]):
    for s in [0.001, 0.002]:
        sample_BTC_copy = sample_BTC.copy()
        result = trading_func(timespan=t, sensitivity=s, elements_to_process=len(sample_BTC_copy), data=sample_BTC_copy)
        sample_BTC_copy = sample_BTC_copy.join(pd.DataFrame(result, columns = ["index", "decision"]).set_index("index")).fillna('wait')
        sample_BTC_copy = calc_profit(sample_BTC_copy)

        sns.lineplot(data = sample_BTC_copy, x = 'tmsp', y = "profit")
        plt.show()

        print(sample_BTC_copy.profit.describe())
        print("Duration", t, "sensitivity", s)

100%|███████████████████████████████████████████████████████████████████████████| 53549/53549 [02:35<00:00, 344.19it/s]


AttributeError: 'DataFrame' object has no attribute 'decision'